In [13]:
"""
Supabase Dataset Retrieval and DataFrame Creation Notebook
==========================================================
This notebook retrieves token snapshot datasets from Supabase storage
and creates a pandas DataFrame with one row per token containing daily data.

Requirements:
pip install supabase pandas python-dotenv requests
"""

# %% [markdown]
# # Supabase Token Dataset Loader
# 
# This notebook:
# 1. Connects to Supabase storage
# 2. Lists all dataset files from the analytics/snapshots folder
# 3. Downloads and parses JSON snapshot files
# 4. Creates a pandas DataFrame with one row per token

'\nSupabase Dataset Retrieval and DataFrame Creation Notebook\n==========================================================\nThis notebook retrieves token snapshot datasets from Supabase storage\nand creates a pandas DataFrame with one row per token containing daily data.\n\nRequirements:\npip install supabase pandas python-dotenv requests\n'

### Import libraries

In [14]:
import os
import json
import pandas as pd
import requests
from datetime import datetime
from typing import List, Dict, Optional
from supabase import create_client, Client
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True


### Configuration

In [22]:
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
BUCKET_NAME = os.getenv("SUPABASE_BUCKET", "monitor-data")
DATASET_DIR_REMOTE = os.getenv("DATASET_DIR_REMOTE", "datasets")

if not SUPABASE_URL or not SUPABASE_KEY:
    raise RuntimeError("❌ Missing SUPABASE_URL or SUPABASE_KEY in environment variables")

# Initialize Supabase client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
print(f"✅ Connected to Supabase: {SUPABASE_URL}")

global _file_cache_headers

✅ Connected to Supabase: https://ldraroaloinsesjoayxc.supabase.co



### Helper Functions

In [16]:
# Download function using signed URLs
def download_json_from_supabase(remote_path: str, local_dataset_root: str = "./dataset") -> Optional[Dict]:
    """
    Download and parse a JSON file from Supabase using a signed URL,
    with conditional GET (ETag/If-Modified-Since) caching.
    
    Saves to and loads from a local 'dataset' folder to avoid refetching.

    Args:
        remote_path: Path to file in Supabase storage 
                     (e.g., 'mint_discovery/2025-01-15/token.json')
        local_dataset_root: The local root directory to save/load cached files.
    
    Returns:
        Parsed JSON as dict, or None if error.
    """
    global _file_cache_headers

    # 1. Determine local save path
    save_path = os.path.join(local_dataset_root, remote_path)
    file_content_bytes: Optional[bytes] = None

    try:
        # 2. Generate a signed URL
        signed_url_response = supabase.storage.from_(BUCKET_NAME).create_signed_url(remote_path, 60)
        signed_url = signed_url_response.get('signedURL')
        
        if not signed_url:
            print(f"❌ Could not generate signed URL for '{remote_path}'")
            # If we can't get a URL, check the local cache as a fallback
            if os.path.exists(save_path):
                print(f"Loading from local cache as fallback: '{save_path}'")
                with open(save_path, "rb") as f:
                    file_content_bytes = f.read()
            else:
                return None  # No URL, no local file
        
        if not file_content_bytes:
            # 3. Prepare headers for conditional GET
            headers = {}
            cached_headers = _file_cache_headers.get(remote_path, {})
            if cached_headers.get('Last-Modified'):
                headers['If-Modified-Since'] = cached_headers['Last-Modified']
            if cached_headers.get('ETag'):
                headers['If-None-Match'] = cached_headers['ETag']

            # 4. Perform the HTTP request
            response = requests.get(signed_url, headers=headers, timeout=15)

            # 5. Handle the response
            if response.status_code == 304:
                # 304 Not Modified
                print(f"File '{remote_path}': No change detected (304 Not Modified).")
                if os.path.exists(save_path):
                    print(f"Loading from local cache: '{save_path}'")
                    with open(save_path, "rb") as f:
                        file_content_bytes = f.read()
                else:
                    # File not modified, but local copy is missing. Force re-download.
                    print(f"File '{remote_path}' not modified, but local file '{save_path}' missing. Forcing re-download.")
                    headers.pop('If-Modified-Since', None)
                    headers.pop('If-None-Match', None)
                    _file_cache_headers.pop(remote_path, None)
                    response = requests.get(signed_url, headers=headers, timeout=15)
                    # Allow to fall through to 200 logic if it's 200 now

            if response.status_code == 200:
                # 200 OK: File is new or has been updated
                print(f"File '{remote_path}': File updated — new data loaded.")
                file_content_bytes = response.content

                # Update our cache with the new 'Last-Modified' and 'ETag' headers
                new_last_modified = response.headers.get('Last-Modified')
                new_etag = response.headers.get('ETag')
                
                new_headers_to_cache = {}
                if new_last_modified:
                    new_headers_to_cache['Last-Modified'] = new_last_modified
                if new_etag:
                    new_headers_to_cache['ETag'] = new_etag
                    
                if new_headers_to_cache:
                    _file_cache_headers[remote_path] = new_headers_to_cache
                    # print(f"File '{remote_path}': Updated cache headers.") # Optional: for more verbose logging

                # Save the new file content locally
                os.makedirs(os.path.dirname(save_path) or ".", exist_ok=True)
                with open(save_path, "wb") as f:
                    f.write(file_content_bytes)
                print(f"Downloaded and saved '{remote_path}' -> '{save_path}'")

            elif response.status_code != 304: # if it wasn't 200 or 304
                # Handle other errors (404 Not Found, 403 Forbidden, 500, etc.)
                print(f"File '{remote_path}': Error fetching file. Status: {response.status_code}, Response: {response.text[:100]}...")
                if os.path.exists(save_path):
                    print(f"Loading from local cache as fallback: '{save_path}'")
                    with open(save_path, "rb") as f:
                        file_content_bytes = f.read()
                else:
                    return None # No file, no cache.

    except requests.exceptions.RequestException as e:
        print(f"File '{remote_path}': Network or request error. {e}")
        if os.path.exists(save_path):
            print(f"Loading from local cache as fallback: '{save_path}'")
            with open(save_path, "rb") as f:
                file_content_bytes = f.read()
    except Exception as e:
        print(f"File '{remote_path}': An unexpected error occurred. {e}")
        if os.path.exists(save_path):
            print(f"Loading from local cache as fallback: '{save_path}'")
            with open(save_path, "rb") as f:
                file_content_bytes = f.read()

    # 6. If we have bytes (from download or cache), parse them
    if file_content_bytes:
        try:
            return json.loads(file_content_bytes.decode('utf-8'))
        except json.JSONDecodeError as e:
            print(f"❌ Failed to parse JSON from '{remote_path}' (local: '{save_path}'): {e}")
            return None
        except UnicodeDecodeError as e:
            print(f"❌ Failed to decode UTF-8 from '{remote_path}' (local: '{save_path}'): {e}")
            return None
    
    # If we reach here, all attempts failed
    print(f"❌ Failed to retrieve or load '{remote_path}'")
    return None

In [17]:

# List files function
def list_dataset_files(pipeline: str = None, date_folder: str = None, limit: int = 1000) -> List[Dict]:
    """
    List all dataset files in Supabase storage.
    
    Args:
        pipeline: Filter by pipeline (e.g., 'mint_discovery', 'mint_alpha')
        date_folder: Filter by date folder (e.g., '2025-01-15')
        limit: Maximum number of files to return
    
    Returns:
        List of file metadata dicts
    """
    try:
        if pipeline and date_folder:
            folder = f"{DATASET_DIR_REMOTE}/{pipeline}/{date_folder}"
        elif pipeline:
            folder = f"{DATASET_DIR_REMOTE}/{pipeline}"
        else:
            folder = DATASET_DIR_REMOTE
        
        files = supabase.storage.from_(BUCKET_NAME).list(
            folder,
            {"limit": limit, "sortBy": {"column": "created_at", "order": "desc"}}
        )
        
        return files if files else []
    except Exception as e:
        print(f"❌ Error listing files in {folder}: {e}")
        return []

In [18]:
def list_all_dataset_files(pipeline: str = None, date_folder: str = None, include_expired: bool = False) -> List[str]:
    """
    Recursively list all dataset JSON files across all date folders.
    
    Args:
        pipeline: Filter by pipeline ('discovery' or 'alpha'). If None, gets both.
        date_folder: Filter by specific date (e.g., '2025-01-15'). If None, gets all dates.
        include_expired: Whether to include files from 'expired_no_label' folder. Default False.
    
    Returns:
        List of full file paths
    """
    all_files = []
    
    try:
        # Define pipelines: 'discovery' and 'alpha' folders
        if pipeline:
            pipelines = [pipeline]
        else:
            pipelines = ['discovery', 'alpha']
        
        print(f"📂 Scanning pipelines: {pipelines}")
        
        # For each pipeline, list date folders
        for pipe in pipelines:
            pipeline_path = f"{DATASET_DIR_REMOTE}/{pipe}"
            
            try:
                # List all folders in this pipeline (these are the daily folders)
                date_folders = supabase.storage.from_(BUCKET_NAME).list(pipeline_path)
                
                print(f"\n  📅 Pipeline '{pipe}' - Found {len(date_folders)} folders")
                
                # Filter date folders if specified
                if date_folder:
                    date_folders = [f for f in date_folders if f.get('name') == date_folder]
                    print(f"     Filtered to date: {date_folder}")
                
                for date_item in date_folders:
                    folder_name = date_item.get('name')
                    if not folder_name:
                        continue
                    
                    # Skip 'expired_no_label' folder unless explicitly requested
                    if folder_name == 'expired_no_label' and not include_expired:
                        print(f"    ⏭️  Skipping: {folder_name}")
                        continue
                    
                    date_path = f"{pipeline_path}/{folder_name}"
                    
                    try:
                        # List JSON files in this date folder
                        files = supabase.storage.from_(BUCKET_NAME).list(date_path)
                        
                        # Filter for JSON files only (files won't have 'id' field or will have metadata)
                        json_files = []
                        for f in files:
                            fname = f.get('name', '')
                            if fname.endswith('.json'):
                                # This is a file, not a folder
                                json_files.append(f)
                        
                        print(f"    📁 {folder_name}: {len(json_files)} JSON files")
                        
                        for file in json_files:
                            full_path = f"{date_path}/{file['name']}"
                            all_files.append(full_path)
                            
                    except Exception as e:
                        print(f"    ❌ Error listing files in '{date_path}': {e}")
                        continue
                        
            except Exception as e:
                print(f"  ❌ Error scanning pipeline '{pipe}': {e}")
                continue
        
        print(f"\n✅ Total files found: {len(all_files)}")
        
        # Show sample of files found
        if all_files:
            print("\n📄 Sample files:")
            for i, file_path in enumerate(all_files[:3]):
                print(f"  {i+1}. {file_path}")
            if len(all_files) > 3:
                print(f"  ... and {len(all_files) - 3} more")
        else:
            print("\n⚠️  No files found. Showing folder structure for debugging:")
            try:
                # Try to list what's actually in the datasets folder
                for pipe in pipelines:
                    pipeline_path = f"{DATASET_DIR_REMOTE}/{pipe}"
                    pipeline_contents = supabase.storage.from_(BUCKET_NAME).list(pipeline_path)
                    print(f"\n📂 '{pipeline_path}' contains {len(pipeline_contents)} items:")
                    for item in pipeline_contents[:10]:  # Show first 10
                        item_name = item.get('name')
                        print(f"  - {item_name}")
                        # Try to peek inside each item
                        try:
                            folder_path = f"{pipeline_path}/{item_name}"
                            folder_contents = supabase.storage.from_(BUCKET_NAME).list(folder_path)
                            json_count = len([f for f in folder_contents if f.get('name', '').endswith('.json')])
                            total_count = len(folder_contents)
                            print(f"    → Contains {total_count} items ({json_count} JSON files)")
                            if json_count > 0:
                                print(f"    → Sample: {folder_contents[0].get('name', 'N/A')}")
                        except Exception as e:
                            print(f"    → Error: {e}")
            except Exception as e:
                print(f"❌ Debug failed: {e}")
        
        return all_files
        
    except Exception as e:
        print(f"❌ Error listing dataset files: {e}")
        return []



#### Data Retrieval and Processing


In [19]:
from typing import Dict

def flatten_snapshot_to_row(snapshot: Dict) -> Dict:
    """
    Flatten a snapshot JSON into a single row dict for DataFrame.
    
    Args:
        snapshot: Snapshot dict loaded from JSON
    
    Returns:
        Flattened dict with all relevant fields
    """
    row = {}
    
    # Snapshot metadata
    row['snapshot_id'] = snapshot.get('snapshot_id')
    row['generated_at_utc'] = snapshot.get('generated_at_utc')

    # --- Inputs ---
    # Define helper variables for nested input objects, using .get('key', {}) 
    # to safely chain .get() calls even if a parent object is missing.
    inputs = snapshot.get('inputs', {})
    signal_data = inputs.get('signal_data', {})
    signal_data_result = signal_data.get('result', {})
    security_data = signal_data_result.get('security', {})
    rugcheck_data = security_data.get('rugcheck', {})
    dexscreener_data = signal_data_result.get('dexscreener', {})
    holiday_check = inputs.get('holiday_check', {})
    
    # Inputs - signal_data
    row['input_signal_data_ts'] = signal_data.get('ts')
    row['input_signal_data_check_type'] = signal_data.get('check_type')
    row['input_signal_data_security'] = signal_data.get('security')

    # Inputs - signal_data.result
    row['input_signal_data_result_mint'] = signal_data_result.get('mint')
    row['input_signal_data_result_checked_at'] = signal_data_result.get('checked_at')
    row['input_signal_data_result_overlap_count'] = signal_data_result.get('overlap_count')
    row['input_signal_data_result_overlap_percentage'] = signal_data_result.get('overlap_percentage')
    row['input_signal_data_result_concentration'] = signal_data_result.get('concentration')
    row['input_signal_data_result_weighted_concentration'] = signal_data_result.get('weighted_concentration')
    row['input_signal_data_result_total_winner_wallets'] = signal_data_result.get('total_winner_wallets')
    row['input_signal_data_result_grade'] = signal_data_result.get('grade')
    row['input_signal_data_result_needs_monitoring'] = signal_data_result.get('needs_monitoring')
    row['input_signal_data_result_skip_reason'] = signal_data_result.get('skip_reason')

    # Inputs - signal_data.result.security
    row['input_security_rugcheck_passed'] = security_data.get('rugcheck_passed')
    row['input_security_probation'] = security_data.get('probation')
    row['input_security_probation_reason'] = security_data.get('probation_reason')

    # Inputs - signal_data.result.security.rugcheck
    row['input_rugcheck_rugged'] = rugcheck_data.get('rugged')
    row['input_rugcheck_top1_holder_pct'] = rugcheck_data.get('top1_holder_pct')
    row['input_rugcheck_holder_count'] = rugcheck_data.get('holder_count')
    row['input_rugcheck_has_authorities'] = rugcheck_data.get('has_authorities')
    row['input_rugcheck_creator_balance'] = rugcheck_data.get('creator_balance')
    row['input_rugcheck_freeze_authority'] = rugcheck_data.get('freeze_authority')
    row['input_rugcheck_mint_authority'] = rugcheck_data.get('mint_authority')
    row['input_rugcheck_transfer_fee_pct'] = rugcheck_data.get('transfer_fee_pct')
    row['input_rugcheck_lp_locked_pct'] = rugcheck_data.get('lp_locked_pct')
    row['input_rugcheck_total_liquidity_usd'] = rugcheck_data.get('total_liquidity_usd')

    # Inputs - signal_data.result.dexscreener
    row['input_dexscreener_current_price_usd'] = dexscreener_data.get('current_price_usd')

    # Inputs - holiday_check
    row['input_holiday_check_is_holiday'] = holiday_check.get('is_holiday')
    # Note: Skipping 'countries' list as it's not a scalar value.

    # --- Features ---
    features = snapshot.get('features', {})
    row['mint'] = features.get('mint')
    row['signal_source'] = features.get('signal_source')
    row['grade'] = features.get('grade')
    row['checked_at_utc'] = features.get('checked_at_utc')
    row['checked_at_timestamp'] = features.get('checked_at_timestamp')
    
    # Time features
    row['time_of_day_utc'] = features.get('time_of_day_utc')
    row['day_of_week_utc'] = features.get('day_of_week_utc')
    row['is_weekend_utc'] = features.get('is_weekend_utc')
    row['is_public_holiday_any'] = features.get('is_public_holiday_any')
    
    # Market features (from original function)
    row['price_usd'] = features.get('price_usd')
    row['fdv_usd'] = features.get('fdv_usd')
    row['liquidity_usd'] = features.get('liquidity_usd')
    row['volume_h24_usd'] = features.get('volume_h24_usd')
    row['price_change_h24_pct'] = features.get('price_change_h24_pct')
    row['pair_created_at_timestamp'] = features.get('pair_created_at_timestamp')
    
    # Security features
    row['rugcheck_risk_level'] = features.get('rugcheck_risk_level')
    row['is_rugged'] = features.get('is_rugged')
    row['has_mint_authority'] = features.get('has_mint_authority')
    row['has_freeze_authority'] = features.get('has_freeze_authority')
    row['creator_balance_pct'] = features.get('creator_balance_pct')
    row['top_10_holders_pct'] = features.get('top_10_holders_pct')
    row['is_lp_locked_95_plus'] = features.get('is_lp_locked_95_plus')
    row['total_lp_locked_usd'] = features.get('total_lp_locked_usd')
    
    # Derived features (from original function)
    row['token_age_at_signal_seconds'] = features.get('token_age_at_signal_seconds')
    
    # --- Finalization ---
    finalization = snapshot.get('finalization', {})
    row['token_age_hours_at_signal'] = finalization.get('token_age_hours_at_signal')
    row['is_new_token'] = finalization.get('is_new_token')
    row['finalize_window_hours'] = finalization.get('finalize_window_hours')
    row['finalize_deadline'] = finalization.get('finalize_deadline')
    row['check_interval_minutes'] = finalization.get('check_interval_minutes')
    row['next_check_at'] = finalization.get('next_check_at')
    row['check_count'] = finalization.get('check_count')
    row['finalization_status'] = finalization.get('finalization_status')
    row['claimed_by'] = finalization.get('claimed_by')
    row['claimed_at'] = finalization.get('claimed_at')
    row['finalized_at'] = finalization.get('finalized_at')
    
    # --- Label ---
    label = snapshot.get('label')
    if label:
        row['label_mint'] = label.get('mint')
        row['label_signal_type'] = label.get('signal_type')
        row['label_symbol'] = label.get('symbol')
        row['label_name'] = label.get('name')
        row['label_entry_price'] = label.get('entry_price')
        row['label_entry_time'] = label.get('entry_time')
        row['label_token_age_hours'] = label.get('token_age_hours')
        row['label_pair_created_at'] = label.get('pair_created_at')
        row['label_tracking_interval_seconds'] = label.get('tracking_interval_seconds')
        row['label_tracking_duration_hours'] = label.get('tracking_duration_hours')
        row['label_tracking_end_time'] = label.get('tracking_end_time')
        row['label_current_price'] = label.get('current_price')
        row['label_current_roi'] = label.get('current_roi')
        row['label_ath_price'] = label.get('ath_price')
        row['label_ath_roi'] = label.get('ath_roi')
        row['label_ath_time'] = label.get('ath_time')
        row['label_status'] = label.get('status')
        row['label_hit_50_percent'] = label.get('hit_50_percent')
        row['label_hit_50_percent_time'] = label.get('hit_50_percent_time')
        row['label_time_to_ath_minutes'] = label.get('time_to_ath_minutes')
        row['label_time_to_50_percent_minutes'] = label.get('time_to_50_percent_minutes')
        row['label_last_price_check'] = label.get('last_price_check')
        row['label_last_successful_price'] = label.get('last_successful_price')
        row['label_consecutive_failures'] = label.get('consecutive_failures')
        row['label_retry_start_time'] = label.get('retry_start_time')
        row['label_final_price'] = label.get('final_price')
        row['label_final_roi'] = label.get('final_roi')
        row['label_tracking_completed_at'] = label.get('tracking_completed_at')
        
        # Fields from original function
        row['label_win_pct'] = label.get('win_pct')
        row['label_max_gain_pct'] = label.get('max_gain_pct')
        row['label_max_drawdown_pct'] = label.get('max_drawdown_pct')
        row['label_final_pnl_pct'] = label.get('final_pnl_pct')
        row['label_duration_hours'] = label.get('duration_hours')
    else:
        # Set all label fields to None if label is missing
        keys = [
            'label_mint', 'label_signal_type', 'label_symbol', 'label_name',
            'label_entry_price', 'label_entry_time', 'label_token_age_hours',
            'label_pair_created_at', 'label_tracking_interval_seconds',
            'label_tracking_duration_hours', 'label_tracking_end_time',
            'label_current_price', 'label_current_roi', 'label_ath_price',
            'label_ath_roi', 'label_ath_time', 'label_status',
            'label_hit_50_percent', 'label_hit_50_percent_time',
            'label_time_to_ath_minutes', 'label_time_to_50_percent_minutes',
            'label_last_price_check', 'label_last_successful_price',
            'label_consecutive_failures', 'label_retry_start_time',
            'label_final_price', 'label_final_roi', 'label_tracking_completed_at',
            'label_win_pct', 'label_max_gain_pct', 'label_max_drawdown_pct',
            'label_final_pnl_pct', 'label_duration_hours'
        ]
        for key in keys:
            row[key] = None
    
    return row

In [20]:

# Load datasets into DataFrame
def load_datasets_to_dataframe(pipeline: str = None, date_folder: str = None, limit: int = None) -> pd.DataFrame:
    """
    Load all dataset files from Supabase and create a pandas DataFrame.
    
    Args:
        pipeline: Filter by pipeline ('discovery' or 'alpha'). None = both.
        date_folder: Filter by specific date folder (e.g., '2025-01-15'). None = all dates.
        limit: Maximum number of files to process (None = all)
    
    Returns:
        pandas DataFrame with one row per token snapshot
    """
    print("🔍 Listing dataset files...")
    file_paths = list_all_dataset_files(pipeline=pipeline, date_folder=date_folder)
    
    if limit:
        file_paths = file_paths[:limit]
        print(f"⚠️ Limited to {limit} files")
    
    if not file_paths:
        print("❌ No dataset files found")
        return pd.DataFrame()
    
    print(f"\n📥 Downloading and processing {len(file_paths)} files...")
    
    rows = []
    errors = 0
    
    for i, file_path in enumerate(file_paths):
        if (i + 1) % 10 == 0:
            print(f"Progress: {i + 1}/{len(file_paths)} files processed...")
        
        snapshot = download_json_from_supabase(file_path)
        
        if snapshot:
            try:
                row = flatten_snapshot_to_row(snapshot)
                rows.append(row)
            except Exception as e:
                print(f"❌ Error processing {file_path}: {e}")
                errors += 1
        else:
            errors += 1
    
    print(f"\n✅ Successfully processed {len(rows)} snapshots ({errors} errors)")
    
    # Create DataFrame
    df = pd.DataFrame(rows)
    
    # Convert timestamps to datetime
    if 'checked_at_utc' in df.columns:
        df['checked_at_utc'] = pd.to_datetime(df['checked_at_utc'])
    if 'generated_at_utc' in df.columns:
        df['generated_at_utc'] = pd.to_datetime(df['generated_at_utc'])
    if 'finalized_at' in df.columns:
        df['finalized_at'] = pd.to_datetime(df['finalized_at'])
    if 'label_tracking_completed_at' in df.columns:
        df['label_tracking_completed_at'] = pd.to_datetime(df['label_tracking_completed_at'])
    
    return df

### Load Data

In [21]:
# Now let's load the data from Supabase into a DataFrame

# %% Load all datasets (adjust parameters as needed)
# Options:
# - Load all: df = load_datasets_to_dataframe()
# - Load discovery only: df = load_datasets_to_dataframe(pipeline='discovery')
# - Load alpha only: df = load_datasets_to_dataframe(pipeline='alpha')
# - Load specific date: df = load_datasets_to_dataframe(date_folder='2025-01-15')
# - Load with limit: df = load_datasets_to_dataframe(limit=100)

# Example: Load first 50 files from both pipelines
df = load_datasets_to_dataframe() #limit=50)

# Example: Load all discovery data
# df = load_datasets_to_dataframe(pipeline='discovery')

# Example: Load specific date from alpha
# df = load_datasets_to_dataframe(pipeline='alpha', date_folder='2025-01-15')

# %% Display DataFrame info
print("\n" + "="*80)
print("DATAFRAME SUMMARY")
print("="*80)
print(f"\nShape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\nColumns: {list(df.columns)}")
print("\nData types:")
print(df.dtypes)
print("\nFirst few rows:")
df.head()

# %% Basic statistics
print("\n" + "="*80)
print("BASIC STATISTICS")
print("="*80)

if not df.empty:
    print(f"\nUnique tokens: {df['mint'].nunique()}")
    print(f"Signal sources: {df['signal_source'].value_counts().to_dict()}")
    print(f"Grade distribution: {df['grade'].value_counts().to_dict()}")
    print(f"Label status distribution: {df['label_status'].value_counts().to_dict()}")
    
    print("\nDate range:")
    print(f"  Earliest: {df['checked_at_utc'].min()}")
    print(f"  Latest: {df['checked_at_utc'].max()}")
    
    print("\nMarket metrics:")
    print(df[['price_usd', 'fdv_usd', 'liquidity_usd', 'volume_h24_usd']].describe())

# %% Export to CSV (optional)
# df.to_csv('token_datasets.csv', index=False)
# print("\n✅ Exported to token_datasets.csv")

# %% [markdown]
# ## Analysis Examples
# 
# Here are some example analyses you can perform:

# %% Example: Filter by pipeline
if not df.empty:
    discovery_df = df[df['signal_source'] == 'discovery']
    alpha_df = df[df['signal_source'] == 'alpha']
    
    print(f"\nDiscovery tokens: {len(discovery_df)}")
    print(f"Alpha tokens: {len(alpha_df)}")

# %% Example: Group by date
if not df.empty and 'checked_at_utc' in df.columns:
    df['date'] = df['checked_at_utc'].dt.date
    daily_counts = df.groupby('date').size()
    print("\nDaily token counts:")
    print(daily_counts)

# %% Example: Analyze labeled vs unlabeled
if not df.empty:
    labeled = df[df['label_status'].notna()]
    unlabeled = df[df['label_status'].isna()]
    
    print(f"\nLabeled tokens: {len(labeled)} ({len(labeled)/len(df)*100:.1f}%)")
    print(f"Unlabeled tokens: {len(unlabeled)} ({len(unlabeled)/len(df)*100:.1f}%)")
    
    if len(labeled) > 0:
        print(f"\nWin rate: {(labeled['label_status'] == 'win').sum() / len(labeled) * 100:.1f}%")
        print(f"Average max gain: {labeled['label_max_gain_pct'].mean():.2f}%")
        print(f"Average final PnL: {labeled['label_final_pnl_pct'].mean():.2f}%")




🔍 Listing dataset files...
📂 Scanning pipelines: ['discovery', 'alpha']

  📅 Pipeline 'discovery' - Found 1 folders
    ⏭️  Skipping: expired_no_label

  📅 Pipeline 'alpha' - Found 6 folders
    📁 2025-11-03: 2 JSON files
    📁 2025-11-05: 1 JSON files
    📁 2025-11-06: 21 JSON files
    📁 2025-11-07: 24 JSON files
    📁 2025-11-08: 9 JSON files
    ⏭️  Skipping: expired_no_label

✅ Total files found: 57

📄 Sample files:
  1. datasets/alpha/2025-11-03/2wTr7CbhVx85JeJjJ4aVsMFJKcvLwDJyfyK3SKd6pump_2025-11-03T14-48-04.181771_00-00.json
  2. datasets/alpha/2025-11-03/5yVT7keCgESd6S523y6EwiVfJ3JoRj8f28JkZt2ipump_2025-11-03T15-03-11.343913_00-00.json
  3. datasets/alpha/2025-11-05/4bcfWqNrGgvEy8NA3kWeCbZr46Qpd175hWbUXtkopump_2025-11-05T04-14-22.642681_00-00.json
  ... and 54 more

📥 Downloading and processing 57 files...
File 'datasets/alpha/2025-11-03/2wTr7CbhVx85JeJjJ4aVsMFJKcvLwDJyfyK3SKd6pump_2025-11-03T14-48-04.181771_00-00.json': An unexpected error occurred. name '_file_cache_headers'

KeyboardInterrupt: 

In [ ]:
df.columns

RangeIndex(start=0, stop=0, step=1)